In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("subho117/fake-news-detection-using-machine-learning")

print("Path to dataset files:", path)

d:\src_git\learn_ds\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


100%|██████████| 41.0M/41.0M [00:04<00:00, 9.70MB/s]

Extracting files...


Path to dataset files: C:\Users\kumar\.cache\kagglehub\datasets\subho117\fake-news-detection-using-machine-learning\versions\1
